Language Translation Using Tesseract and Open AI

In [26]:
#import the necessary libraries
import cv2
import pytesseract
import openai
import tkinter as tk #GUI library
import os
import shutil
from langdetect import detect
from gtts import gTTS
from PIL import Image, ImageTk

In [27]:
#constant.py file has my Open AI API Key
import constant

In [28]:
#Creating a folder named temp

folder_name = 'temp'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
else:
    shutil.rmtree(folder_name)
    os.makedirs(folder_name)

In [29]:
original_image = 'sample_text/text-malayalam.png'

In [30]:
#Load the image
image = cv2.imread(original_image)

#Resize the image
width = 350 
height = 250

dim = (width,height) #dim tuple holds width and height values

resized_img = cv2.resize(image, dim, interpolation= cv2.INTER_AREA)


#save the resized image

cv2.imwrite('temp\img-resized.jpg', resized_img)


True

In [31]:
#Convert image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#Image thresholding and contour detection
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] #thresh holds the binary img

#Find contours in a binary image
contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)

cv2.imwrite('temp/contour.jpg', image)


True

In [32]:
#configuring tesseract for multilingual language processing

config = "-l eng+deu+kor+rus+chi_sim+hin+mal"

In [33]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [34]:
#Extracting text from image
text = pytesseract.image_to_string(thresh, config=config)
text

'ഇന്ത്യനു ഭരണവടന\nആമുഖം\n\nന൯മ്മശ, ഇന്ത്യയിലെ ജനങ്ങള്\u200d, ഇന്ത്യയെ ഒരു\nപരമാധികാര, സ്ഥിരിസമത്യ, മതന്ദിരപേക്ഷ, ജനാധിപത്യ\nറികങഞ്ലിക്കായി സംവിധാനം ചെയ്യുന്നതിനും ഭാരതത്തിലെ\nഎല്ലാ പയരര്\u200dക്കും സാമൂഹികവും സാവ്പത്തികവും\nരാഷ്ട്രീയവുമായ നീതി, ചിന്ത, ആശയാവിഷ്\u200cകാരം, വിശ്വാസം,\nഭക്തി, ആരാധന എന്നവ ള്ള സ്വാത,\nസ്ഥാനമാനങ്ങള്\u200d, അവസരങ്ങൾ എന്നിവയിലുള്ള സമത്വം\nഎന്നിവ ഉറഭ്യവരുത്തുന്നുതിരും വൃക്തിയുടെ അന്തസ്സും\nരാഷ്ട്രത്തിന്റെ ഐക്യവും അഖണ്ഡതയും ഉറയ്യവരുത്തുണ്ന\nസാഹോരര്യം എല്ലാവരിലും വളരത്തുന്നുതിനും ദുഡടനിശ്ചയം\nചെയ്തു കൊണ്ട്\u200c നമ്മുടെ ഭരണഘടനാസഭയില്\u200dവച്ച്\u200c, 1949\nനവംബറിന്റെ ഈ ഇരുപത്തിയാഠാം ദിവസം, ഈ\nഭരണഘടനയെ അംഗീകരിക്കുകയും നിയമമാക്കുകയും\nനമുക്കായിത്തന്നെ സമര്\u200dഭിക്കുകയും ചെയ്യുന്നു.\n'

In [35]:
# Function to detect the language of the given text
def detect_language(text):
    return detect(text)

In [36]:
lan = detect_language(text)
lan

'ml'

In [37]:
#Speech to text
tts = gTTS(text=text, lang=lan)
tts.save("output.mp3")
os.system("start output.mp3")

0

Translation Using OpenAI

In [38]:
#Set the Open API Key
openai.api_key = constant.api_key
#Now you can use the openai library to make API requests

In [39]:
#Function to translate text using Openai API

def translate_text(text, target_language):
    model_engine = "text-davinci-003"
    response = openai.Completion.create(
        engine=model_engine,
        prompt=f"translate '{text}' to {target_language}",
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    
    translated_text = response["choices"][0]["text"].strip()
    #print(response)
    return translated_text




In [40]:
input_text = text
target_language = input("Enter your target language:")

In [41]:
translation = translate_text(input_text, target_language)
translation

'Preamble\n\nWe, the people of India, having solemnly resolved to constitute India into a Sovereign, Socialist, Secular, Democratic Republic and to secure to all its citizens: Justice, social, economic and political; Liberty of thought, expression, belief, faith and worship; Equality of status and of opportunity; and to promote among them all Fraternity assuring the dignity of the individual and the unity and integrity of the Nation; In our Constituent Assembly this twenty-sixth day of November, 1949, do hereby adopt, enact and give to ourselves this Constitution.'

In [42]:
lan = detect_language(translation)
lan

'en'

In [43]:

#Speech to text
tts = gTTS(text=translation, lang=lan)
tts.save("output1.mp3")
os.system("start output1.mp3")


0

Creating Tkinter Application

In [44]:
#create the root window
root = tk.Tk()

#prepare images for display
img_input = Image.open('temp\img-resized.jpg')

img_1 = ImageTk.PhotoImage(img_input)

contour = cv2.imread('temp\contour.jpg')

resized_contour = cv2.resize(contour, dim, interpolation = cv2.INTER_AREA)

cv2.imwrite('temp\contour-resized.jpg', resized_contour)

img_contour = Image.open('temp\contour-resized.jpg')

img_2 = ImageTk.PhotoImage(img_contour)



# GUI

root.title("Language Translation")


image_origin = tk.Label(root, image=img_1)

image_origin.grid(row=0, column=0) #use the grid geometry manager to place the image labels in the specified rows and columns on the GUI

image_segmen = tk.Label(root, image=img_2)


image_segmen.grid(row=0, column=1)


label_text_detected = tk.Label(root, text="Detected Text : \n" + text)

label_text_detected.grid(row=1, column=0)


#control the appearance of the translated text
font_size = 10
window_width = 800


characters_per_line = window_width // font_size

text_lines = [translation[i:i+characters_per_line] for i in range(0, len(translation), characters_per_line)]


label_text_translate = tk.Label(root, text="Translated Text : \n" + "\n".join(text_lines), font=("Helvetica", font_size))
label_text_translate.grid(row=1, column=1)

# Run GUI
root.mainloop()

PDF Translation

In [45]:
import platform
from tempfile import TemporaryDirectory
from pathlib import Path
from pdf2image import convert_from_path

In [46]:
path_to_poppler_exe = Path(r"C:\Program Files\poppler-23.08.0\Library\bin")
out_directory = Path(r"F:\DL Projects\OpenCVProjects\Translation")

In [47]:
pdf = Path(r"F:\DL Projects\OpenCVProjects\Translation\sample_text\Hitler.pdf")

In [48]:
text_file = out_directory / Path("Out_text.txt")

In [49]:
image_file_list = []

In [50]:
def main():

    with TemporaryDirectory() as tempdir:
        # Create a temporary directory to hold our temporary images
 
        if platform.system() == "Windows":
            pdf_pages = convert_from_path(
                pdf, 500, poppler_path=path_to_poppler_exe  ## Read in the PDF file at 500 DPI
            )

        else:
            pdf_pages = convert_from_path(pdf, 500)
        
 
        # Iterate through all the pages stored above
        for page_enumeration, page in enumerate(pdf_pages, start=1):
            # enumerate() "counts" the pages for us.
 
            # Create a file name to store the image
            filename = f"{tempdir}\page_{page_enumeration:03}.jpg"
 
            # Declaring filename for each page of PDF as JPG
            # For each page, filename will be:
            # PDF page 1 -> page_001.jpg
            # PDF page 2 -> page_002.jpg
            # ....
            
 
            # Save the image of the page in system
            page.save(filename, "JPEG")
            image_file_list.append(filename)
 
        
        #Recognizing text from the images using OCR
        
 
        with open(text_file, "a", encoding = 'utf-8') as output_file:
            # Open the file in append mode so that
            # All contents of all images are added to the same file
 
            # Iterate from 1 to total number of pages
            for image_file in image_file_list:

                # Recognize the text as string in image using pytesserct
                text = str(((pytesseract.image_to_string(Image.open(image_file)))))
                target_lan = input("Enter the target language:")
                translated_text = translate_text(text, target_lan)
                
                #processing the translated text
                translated_text = translated_text.replace("-\n", "")
                print(translated_text)

                words = translated_text.split()
                print(words)
                words_per_line = 8
                lines = [words[i:i+words_per_line] for i in range(0, len(words), words_per_line)]
                print(lines)
                for line in lines:
                    line_text = ' '.join(line)  # Join words to form a line

                    # Finally, write the processed text to the file.
                    output_file.write(line_text + '\n')

            
            # the file is closed after writing all the text.
            # At the end of the with .. tempdir block, the
            # TemporaryDirectory() we're using gets removed!       
   
     
if __name__ == "__main__":
      
    main()

अडोल्फ हिटलर एक जर्मन राजनीतिज्ञ थे, जो 1933 से 1945 तक जर्मनी के राष्ट्रपति और डिक्टेटर के रूप में शासित किया गया था। वह राष्ट्रवादी जर्मन श्रमिक पार्टी (NSDAP) को नेतृत्व किया और इतिहास के सबसे भयानक दौर को - दूसरे विश्व युद्ध और हॉलोकाउस।

हिटलर 1889 में ओस्ट्रिया-युग्रेन के ब्राउनौ अम इन में पैदा हुए। उन्होंने प्रारंभिक 1920 के दशक में NSDAP में शामिल हो गए और जल्द ही प्रभाव प्राप्त किया। 1923 में एक विफल क्रांति प्रयास के बाद उन्हें एक छोटे से जेल सज़ा मिली। इस समय में उन्होंने अपनी किताब "मेरी कम्प्फ" लिखी, जिसमें उनके रास्त्रवादी और ऐंटीसेमिटिक दृष्टिकोण व्यक्त किए गए।

1933 में शासन ग्रस्त होने के बाद, हिटलर ने जर्मनी को एक टोटलिटेरियन देश में रूपांतरित करने शुरू कि
['अडोल्फ', 'हिटलर', 'एक', 'जर्मन', 'राजनीतिज्ञ', 'थे,', 'जो', '1933', 'से', '1945', 'तक', 'जर्मनी', 'के', 'राष्ट्रपति', 'और', 'डिक्टेटर', 'के', 'रूप', 'में', 'शासित', 'किया', 'गया', 'था।', 'वह', 'राष्ट्रवादी', 'जर्मन', 'श्रमिक', 'पार्टी', '(NSDAP)', 'को', 'नेतृत्व', 'किया', 'और', 'इतिहास', 'के', 'सबसे', 'भयानक', 'दौ